[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ishandandekar/Looking-Fruit/blob/main/Looking_Fruit_nbk.ipynb)

# Looking_Fruit

👋 Hello and welcome to the **Looking_Fruit** notebook. In this notebook I try to replicate the [Fruits-360](https://www.researchgate.net/publication/321475443_Fruit_recognition_from_images_using_deep_learning) research paper. In this paper, researchers have tried to classify images of **131** fruits and vegetables. The data used for these modelling experiments is provided by the paper researchers themselves.

In [ ]:
# Check for GPU
!nvidia-smi -L

## Step 0: Defining the problem


**Objective:**  
To classify the images of various fruits and vegetables with best f1-score.  

**Files:**
- *Train*: This folder contains folders labelled as fruit's/vegetable's name. These subfolders contain images of the respective fruit/vegetable. This folder will be used for training purpose.
- *Test*: This folder contains folders labelled as fruit's/vegetable's name. These subfolders contain images of the respective fruit/vegetable. This folder will be used for testing purpose.

## Step 1: Getting the data
The data used for this project is publicly available on [Kaggle](https://www.kaggle.com/datasets/ishandandekar/fruitimagedataset).

- Use Kaggle's API to download the data into Colab.
- Get utility functions to help in future.
- Configure data files to read using Python.


In [ ]:
# Getting the helper functions script
!wget https://raw.githubusercontent.com/ishandandekar/Looking_Fruit/main/scripts/helper_functions.py

# Get the necessary functions from the python script
from helper_functions import plot_loss_curves, unzip_data, create_model_checkpoint, create_early_stopping, get_metrics

About these helper functions (see documentation for better description):
* **plot_loss_curves** : Plots a line graph (using matplotlib) to see changes in loss and accuracy during training.
* **unzip_data** : Unzips a zip file to a directory
* **create_model_checkpoint** : Creates a model checkpoint with only best weights (monitored over validation loss).
* **create_early_stopping** : Creates an early stopping callback to prevent overfitting.
* **get_metrics** : Returns a dictionary with classification metrics.

In [ ]:
# Install the kaggle library
!pip install -q kaggle

# Upload the Kaggle API keys
from google.colab import files
files.upload()

!mkdir ~/.kaggle

# Copy the json file to the folder
!cp kaggle.json ~/.kaggle

# Change permissions for keys to work with the Kaggle API
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d ishandandekar/fruitimagedataset

# Unzip data
unzip_data('fruitimagedataset.zip', data_dir="raw")

## Step 2: Know more about the data

- Get the statistics about the data.
- Check if the labels are imbalanced.
- Visualize random samples in data.
- (*If required*) Trim data.
- (*If required*) Preprocess the data.
- Make data processing faster using `ImageDataGenerator`.

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall

In [ ]:
# Setting up necessary variables
TRAIN_PATH = '/content/raw/train/train/'
TEST_PATH = '/content/raw/test/test/'
RAW_DATA_PATH = '/content/raw'
MODEL_VERBOSE = 1

In [ ]:
# Checking the number of classes
classes = []

for dirname, _, filenames in os.walk(RAW_DATA_PATH):
    if dirname.startswith(f"{TRAIN_PATH}"):
        classes.append(dirname[len(TRAIN_PATH):])

print(f"Total number of classes to deal with: {len(classes)}")

In [ ]:
# Show random classes present in the dataset
list_of_five_random_labels = random.sample(classes,5)
list_of_five_random_labels

In [ ]:
# Number of images of each fruit/vegetable as a pandas.DataFrame

# List to append the count of images
number_of_images_train = []

for label in classes:
    path = f'{TRAIN_PATH}/{label}'
    count = len(os.listdir(path))
    number_of_images_train.append(count)

train_image_count_df = pd.DataFrame({"Label":classes,"Number of Images":number_of_images_train})

# To view first 10 rows of the dataframe
train_image_count_df.head(10)

In [ ]:
# Label with most number of images
print(f"Label with most number of images:")
print(train_image_count_df.sort_values("Number of Images",ascending=False).head(1))
print(f"Label with least number of images:")
print(train_image_count_df.sort_values("Number of Images",ascending=True).head(1))

In [ ]:
# Show random sample from training of a random fruit/vegetable

random_label = random.choice(classes)
sample_path = f'{TRAIN_PATH}/{random_label}'
random_image= random.choice(os.listdir(sample_path))
random_image_path = f'{sample_path}/{random_image}'

img = mpimg.imread(random_image_path)
imgplot = plt.imshow(img)
plt.axis(False)
plt.title(f'{random_label}')
plt.show()

In [ ]:
# Making new directories for trimmed training data
os.mkdir("data")
os.mkdir("data/train")

In [ ]:
# Trimming data as there are lot of training examples

TRAIN_IMAGES_COUNT = 100

for label in classes:
  path = f'{TRAIN_PATH}/{label}'
  img_files = os.listdir(path)
  random_count_images = random.sample(img_files, TRAIN_IMAGES_COUNT)
  new_path_for_label = f"/content/data/train/{label}"
  os.mkdir(new_path_for_label)

  for img_file in random_count_images:
    shutil.move(f"{path}/{img_file}", new_path_for_label)

In [ ]:
# Image size has been specified in the research paper
IMAGE_SIZE = (100,100)
BATCH_SIZE = 32
TRIMMED_TRAIN_PATH = '/content/data/train'
TEST_PATH = '/content/raw/test/test'

# To make data processing faster
train_data=tf.keras.preprocessing.image_dataset_from_directory(directory=TRIMMED_TRAIN_PATH,
                                                               image_size=IMAGE_SIZE,
                                                               label_mode="categorical",
                                                               batch_size=BATCH_SIZE)

test_data=tf.keras.preprocessing.image_dataset_from_directory(directory=TEST_PATH,
                                                               image_size=IMAGE_SIZE,
                                                               label_mode="categorical",
                                                               batch_size=BATCH_SIZE)

In [ ]:
# See an example of a batch of data and checking pixel values
for images,labels in train_data.take(1):
  for image in images:
    print(f"Shape of an image: {images.shape}")
    print(f"Maximum in an image: {tf.math.reduce_max(images)}")
    break
  break

## Step 3: Describing modelling experiments

- This notebook contains 7 models built to get the best **f1-score** on the test dataset. These models also include the models made by the researchers themselves.  
- Models to be made:
  1. **Model 0** : simple model with fully connected multiple Dense layers; this model acts as a baseline.
  1. **Model 1** : multiple pairs of CNN and MaxPool layers with a Flatten layer and Dense layer in the end for classification.
  1. **Model 2** : exact same model that researchers used in their program script.
  1. **Model 3** : uses **ResNet50** as its base, using transfer learning.
  1. **Model 4** : uses **EfficientNetB0** under the hood unlike previous model
  1. **Model 5** : uses a **fine-tuned ResNet50** as its base.
  1. **Model 6** : uses a **fine-tuned EfficientNetB0** under its hood
- Get classification metrics for each model.


In [ ]:
# Directory to save images of models' architectures
!mkdir model_architectures

#### Model 0 
* **Layers:** 4 Dense layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 10
* **Callbacks:** `ModelCheckpoint`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
model_0 = tf.keras.Sequential([
    layers.Flatten(input_shape=(100,100,3)),
    layers.Input(shape=INPUT_SHAPE),
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(131, activation="softmax")
], name="model_0_dense")

# 2. Compile the model
model_0.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_0.summary()

# 4. Fit the model
history_model_0 = model_0.fit(train_data,
                              epochs=10,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_0.name)])

In [ ]:
# Save model's architecture as an image
plot_model(model_0, to_file='/content/model_architectures/model_0.png')

# About the model history
plot_loss_curves(history_model_0)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_0_dense"
model_0.load_weights(best_weights_path)

# Evaluate on test set
loss_0, accuracy_0, precision_0, recall_0 = model_0.evaluate(test_data)

In [ ]:
# Unravel dataset
y_labels = []
for images,labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())
y_labels = np.array(y_labels).reshape(-1)

In [ ]:
# Getting a metrics report
model_0_metrics = get_metrics(loss_0, accuracy_0, precision_0, recall_0)
model_0_metrics

#### Model 1 
* **Layers:** Multiple Conv2D layers paired with MaxPool layers with a Dense layer at the end
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 25
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
model_1 = tf.keras.Sequential([
     tf.keras.layers.Conv2D(filters=10,
                            kernel_size=3,
                            activation="relu",
                            input_shape=INPUT_SHAPE),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.MaxPool2D(pool_size=2,
                               padding="valid"),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.Conv2D(10,3,activation="relu"),
     tf.keras.layers.MaxPool2D(2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(64, activation="relu"),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(131,activation="softmax")
], name="model_1_Conv2D")

# 2. Compile the model
model_1.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_1.summary()

# 4. Fit the model
history_model_1 = model_1.fit(train_data,
                              epochs=25,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_1.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_1, to_file='/content/model_architectures/model_1.png')

# About the model history
plot_loss_curves(history_model_1)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_1_Conv2D"
model_1.load_weights(best_weights_path)

# Evaluate on test set
loss_1, accuracy_1, precision_1, recall_1 = model_1.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_1_metrics = get_metrics(loss_1, accuracy_1, precision_1, recall_1)
model_1_metrics

#### Model 2 (Researchers' best model)
* **Layers:** Multiple layer model including Conv2D, MaxPool, Dense, Dropouts and Flatten
* **Optimizer:** Adadelta (with learning rate as 0.1)
* **Loss:** Cross entropy
* **Epochs:** 25
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model (taken straight out of their code base)
inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
x = layers.Conv2D(16, (5, 5), strides=(1, 1), padding='same', name='conv1')(inputs)
x = layers.Activation('relu', name='conv1_relu')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool1')(x)
x = layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same', name='conv2')(x)
x = layers.Activation('relu', name='conv2_relu')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool2')(x)
x = layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same', name='conv3')(x)
x = layers.Activation('relu', name='conv3_relu')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool3')(x)
x = layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same', name='conv4')(x)
x = layers.Activation('relu', name='conv4_relu')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), padding='valid', name='pool4')(x)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu', name='fcl1')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu', name='fcl2')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(131, activation='softmax', name='predictions')(x)
model_2 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_2_best_on_paper")

# 2. Compile the model
model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1),
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_2.summary()

# 4. Fit the model
history_model_2 = model_2.fit(train_data,
                              epochs=25,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_2.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_2, to_file='/content/model_architectures/model_2.png')

# About the model history
plot_loss_curves(history_model_2)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_2_best_on_paper"
model_2.load_weights(best_weights_path)

# Evaluate on test set
loss_2, accuracy_2, precision_2, recall_2 = model_2.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_2_metrics = get_metrics(loss_2, accuracy_2, precision_2, recall_2)
model_2_metrics

#### Model 3
* **Layers:** Using ResNet and various other layers to classify images
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 25
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model (taken straight out of their code base)
base_resnet_model = tf.keras.applications.resnet50.ResNet50(include_top=False)
base_resnet_model.trainable = False
inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
scale = layers.Rescaling(scale=1./255)(inputs)
x = base_resnet_model(scale)
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = layers.Dense(131,activation="softmax",name="output_layer")(x)
model_3 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_3_freezed_ResNet")

# 2. Compile the model
model_3.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_3.summary()

# 4. Fit the model
history_model_3 = model_3.fit(train_data,
                              epochs=25,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_3.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_3, to_file='/content/model_architectures/model_3.png')

# About the model history
plot_loss_curves(history_model_3)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_3_freezed_ResNet"
model_3.load_weights(best_weights_path)

# Evaluate on test set
loss_3, accuracy_3, precision_3, recall_3 = model_3.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_3_metrics = get_metrics(loss_3, accuracy_3, precision_3, recall_3)
model_3_metrics

#### Model 4 
* **Layers:** Using EfficientNetB0 (with freezed layers) and various other layers for classification
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 25
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
base_efficient_net_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_efficient_net_model.trainable=False
inputs= layers.Input(shape=INPUT_SHAPE, name="input_layer")
x = base_efficient_net_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs=tf.keras.layers.Dense(131,activation="softmax",name="output_layer")(x)
model_4 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_4_freezed_EfficientNet")

# 2. Compile the model
model_4.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_4.summary()

# 4. Fit the model
history_model_4 = model_4.fit(train_data,
                              epochs=25,
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_4.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_4, to_file='/content/model_architectures/model_4.png')

# About the model history
plot_loss_curves(history_model_4)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_4_freezed_EfficientNet"
model_4.load_weights(best_weights_path)

# Evaluate on test set
loss_4, accuracy_4, precision_4, recall_4 = model_4.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_4_metrics = get_metrics(loss_4, accuracy_4, precision_4, recall_4)
model_4_metrics

#### Model 5
* **Layers:** Fine-tuned ResNet as the base with multiple other layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 30, with `initial_epoch` set to the model 3's last epoch
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
base_resnet_model = tf.keras.applications.resnet50.ResNet50(include_top=False)
base_resnet_model.trainable = False

# Setting the base model to train (only on the last 10 layers)
LAYERS_TRAINABLE = 10
for layer in base_resnet_model.layers[-LAYERS_TRAINABLE:]:
  layer.trainable = True

inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
scale = layers.Rescaling(scale=1./255)(inputs)
x = base_resnet_model(scale)
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = layers.Dense(131,activation="softmax",name="output_layer")(x)
model_5 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_5_finetuned_ResNet")

# 2. Compile the model
model_5.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_5.summary()

# 4. Fit the model
history_model_5 = model_5.fit(train_data,
                              epochs=30,
                              initial_epoch=history_model_3.epoch[-1],
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_5.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_5, to_file='/content/model_architectures/model_5.png')

# About the model history
plot_loss_curves(history_model_5)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_5_finetuned_ResNet"
model_5.load_weights(best_weights_path)

# Evaluate on test set
loss_5, accuracy_5, precision_5, recall_5 = model_5.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_5_metrics = get_metrics(loss_5, accuracy_5, precision_5, recall_5)
model_5_metrics

#### Model 6
* **Layers:** Fine-tuned EffificientNetB0 with other various layers
* **Optimizer:** Adam
* **Loss:** Cross entropy
* **Epochs:** 30, with `initial_epoch` set as model 4's last epoch
* **Callbacks:** `ModelCheckpoint`, `EarlyStopping`
* **Validation data:** available

In [ ]:
# 0. Set random seed
tf.random.set_seed(42)
INPUT_SHAPE = (100,100,3)

# 1. Create the model
base_efficient_net_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_efficient_net_model.trainable=False

# Setting the base model to train (only on the last 10 layers)
LAYERS_TRAINABLE = 10
for layer in base_efficient_net_model.layers[-LAYERS_TRAINABLE:]:
  layer.trainable = True

inputs= layers.Input(shape=INPUT_SHAPE, name="input_layer")
x = base_efficient_net_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs=tf.keras.layers.Dense(131,activation="softmax",name="output_layer")(x)
model_6 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_6_finetuned_EfficientNet")

# 2. Compile the model
model_6.compile(loss="categorical_crossentropy",
                optimizer="Adam",
                metrics=[CategoricalAccuracy(), Precision(), Recall()])

# 3. Get the summary
model_6.summary()

# 4. Fit the model
history_model_6 = model_6.fit(train_data,
                              epochs=30,
                              initial_epoch=history_model_4.epoch[-1],
                              validation_data=test_data,
                              validation_steps=len(test_data)*0.15,
                              verbose=MODEL_VERBOSE,
                              callbacks=[create_model_checkpoint(model_name=model_6.name), create_early_stopping()])

In [ ]:
# Save model's architecture as an image
plot_model(model_6, to_file='/content/model_architectures/model_6.png')

# About the model history
plot_loss_curves(history_model_6)

In [ ]:
# Loading in the best weights
best_weights_path = "/content/model_experiments/model_6_finetuned_EfficientNet"
model_6.load_weights(best_weights_path)

# Evaluate on test set
loss_6, accuracy_6, precision_6, recall_6 = model_6.evaluate(test_data)

In [ ]:
# Getting a metrics report
model_6_metrics = get_metrics(loss_6, accuracy_6, precision_6, recall_6)
model_6_metrics

## Step 4: Compare results and conclude experiments
- Use graphs and matrices to visualize results.
- (*Optional*) Tune hyperparameters of the best model.
- Compare best models results with researchers best model.
- Save and export the models (in .h5 format).


In [ ]:
# Dataframe of results
results_df = pd.DataFrame({"model_0_dense": model_0_metrics,
                           "model_1_Conv2D": model_1_metrics,
                           "model_2_best_on_paper": model_2_metrics,
                           "model_3_freezed_ResNet": model_3_metrics,
                           "model_4_freezed_EfficientNet": model_4_metrics,
                           "model_5_finetuned_ResNet": model_5_metrics,
                           "model_6_finetuned_EfficientNet": model_6_metrics}).T

In [ ]:
results_df[["Accuracy", "Precision", "Recall"]].plot(kind="bar", figsize=(14,7), title="Metrics of various models", ylabel="Metrics", xlabel="Models");

In [ ]:
# Creating directory to store all the models
!mkdir saved_models

# Saving models
model_0.save("saved_models/model_0.h5")
model_1.save("saved_models/model_1.h5")
model_2.save("saved_models/model_2.h5")
model_3.save("saved_models/model_3.h5")
model_4.save("saved_models/model_4.h5")
model_5.save("saved_models/model_5.h5")
model_6.save("saved_models/model_6.h5")